## **Federated Learning for attack detection: 5 nodes sharing gradients: FedAVG**

IDs from this file = **id7xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [16]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [5]:
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')
test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_14900/3457440085.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_14900/3457440085.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')


In [6]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)
xcomplete, ycomplete = preprocessing(test_complete)

### Experiments with datasets

#### Dataset Filt5A

In [7]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part5.csv', low_memory=False)

In [8]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [9]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id700.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [10]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [11]:
global_model = build_model((24,1,1))

In [12]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [13]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [14]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id700.hdf5')

Epoch 1/5
31/31 [==============================] - 7s 228ms/step - loss: 0.3873 - accuracy: 0.9344 - val_loss: 1.7899 - val_accuracy: 0.6197
Epoch 2/5
31/31 [==============================] - 7s 213ms/step - loss: 0.0630 - accuracy: 0.9916 - val_loss: 1.0702 - val_accuracy: 0.6200
Epoch 3/5
31/31 [==============================] - 7s 221ms/step - loss: 0.0310 - accuracy: 0.9919 - val_loss: 0.6936 - val_accuracy: 0.6360
Epoch 4/5
31/31 [==============================] - 7s 215ms/step - loss: 0.0269 - accuracy: 0.9921 - val_loss: 0.6353 - val_accuracy: 0.6487
Epoch 5/5
31/31 [==============================] - 7s 217ms/step - loss: 0.0256 - accuracy: 0.9921 - val_loss: 0.5572 - val_accuracy: 0.6683
Epoch 1/5
31/31 [==============================] - 7s 228ms/step - loss: 0.0422 - accuracy: 0.9899 - val_loss: 0.5192 - val_accuracy: 0.7309
Epoch 2/5
31/31 [==============================] - 7s 219ms/step - loss: 0.0268 - accuracy: 0.9907 - val_loss: 0.5183 - val_accuracy: 0.6733
Epoch 3/5
31/

In [15]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.057443179190158844, 0.2938941717147827, 0.09973163902759552], [0.06505249440670013, 0.34281834959983826, 0.10086438804864883], [0.07566161453723907, 0.5321468114852905, 0.13242985308170319]]
Accuracy for iterations:  [[0.9707192778587341, 0.8688331246376038, 0.94956374168396], [0.9721215963363647, 0.83626389503479, 0.9532325863838196], [0.9716760516166687, 0.7570115327835083, 0.9486151337623596]]
F1 for iterations:  [[0.9706688853212286, 0.8649817140968488, 0.9494720777638193], [0.9720795342963164, 0.8293219364694641, 0.9531637020621871], [0.9716245405472854, 0.734246008100034, 0.9485102660927153]]
Precision for iterations:  [[0.971532954468529, 0.8874553006598564, 0.9528482190249333], [0.9727807229073139, 0.8641774303355073, 0.9559134625704929], [0.9725716710212223, 0.8203151931332536, 0.9522984846048763]]
Recall for iterations:  [[0.9707192731309708, 0.8688331032478384, 0.9495637132155487], [0.9721215928014257, 0.8362638959529173, 0.9532325677098954], [0.971

#### Dataset Filt5B

In [17]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part5.csv', low_memory=False)

In [18]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [19]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id701.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [20]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [21]:
global_model = build_model((24,1,1))

In [22]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [23]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [24]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id701.hdf5')

Epoch 1/5
31/31 [==============================] - 6s 178ms/step - loss: 0.3705 - accuracy: 0.9542 - val_loss: 2.2033 - val_accuracy: 0.5496
Epoch 2/5
31/31 [==============================] - 6s 203ms/step - loss: 0.0624 - accuracy: 0.9916 - val_loss: 1.3871 - val_accuracy: 0.5527
Epoch 3/5
31/31 [==============================] - 6s 201ms/step - loss: 0.0297 - accuracy: 0.9923 - val_loss: 0.8468 - val_accuracy: 0.5572
Epoch 4/5
31/31 [==============================] - 6s 187ms/step - loss: 0.0261 - accuracy: 0.9923 - val_loss: 0.8293 - val_accuracy: 0.5610
Epoch 5/5
31/31 [==============================] - 6s 179ms/step - loss: 0.0247 - accuracy: 0.9924 - val_loss: 0.7090 - val_accuracy: 0.5773
Epoch 1/5
31/31 [==============================] - 6s 180ms/step - loss: 0.0344 - accuracy: 0.9907 - val_loss: 0.4736 - val_accuracy: 0.7157
Epoch 2/5
31/31 [==============================] - 6s 185ms/step - loss: 0.0242 - accuracy: 0.9917 - val_loss: 0.5484 - val_accuracy: 0.6482
Epoch 3/5
31/

In [25]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.12972553074359894, 0.26845505833625793, 0.18629196286201477], [0.0925874412059784, 0.24309733510017395, 0.1514696329832077], [0.10275402665138245, 0.24451933801174164, 0.1729889214038849]]
Accuracy for iterations:  [[0.923982560634613, 0.8365000486373901, 0.8911271095275879], [0.9501227140426636, 0.8805129528045654, 0.9176321029663086], [0.9420374631881714, 0.88343745470047, 0.9093179106712341]]
F1 for iterations:  [[0.9232563832924529, 0.8295778888827323, 0.889908856084678], [0.9498869503995601, 0.8775831881849809, 0.9171320298536756], [0.9416874732802485, 0.8806837182262405, 0.9086378332472831]]
Precision for iterations:  [[0.9325674107935139, 0.8643866622617569, 0.9092398729715513], [0.9536519058631838, 0.8954091223603107, 0.9279618345403645], [0.9468958940466878, 0.8976967067608941, 0.9218783038238343]]
Recall for iterations:  [[0.9239825878640918, 0.8365000363292887, 0.8911270916306873], [0.9501227029711649, 0.8805129695560561, 0.917632124099352], [0.9420

#### Dataset Filt5C

In [26]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part5.csv', low_memory=False)

In [27]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [28]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id702.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [29]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [30]:
global_model = build_model((24,1,1))

In [31]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [32]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [33]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id702.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 190ms/step - loss: 0.5748 - accuracy: 0.7443 - val_loss: 1.5561 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 173ms/step - loss: 0.2592 - accuracy: 0.9027 - val_loss: 2.9254 - val_accuracy: 0.1332
Epoch 3/5
16/16 [==============================] - 3s 163ms/step - loss: 0.0986 - accuracy: 0.9893 - val_loss: 4.1191 - val_accuracy: 0.1339
Epoch 4/5
16/16 [==============================] - 2s 144ms/step - loss: 0.0616 - accuracy: 0.9893 - val_loss: 2.5636 - val_accuracy: 0.1343
Epoch 5/5
16/16 [==============================] - 2s 144ms/step - loss: 0.0410 - accuracy: 0.9892 - val_loss: 1.4367 - val_accuracy: 0.1619
Epoch 1/5
14/14 [==============================] - 2s 143ms/step - loss: 0.0237 - accuracy: 0.9950 - val_loss: 0.8711 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 2s 151ms/step - loss: 0.0207 - accuracy: 0.9950 - val_loss: 0.6643 - val_accuracy: 0.7278
Epoch 3/5

In [34]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.08284654468297958, 0.15706931054592133, 0.09326823055744171], [0.06087246164679527, 0.15701165795326233, 0.06732453405857086], [0.05067482963204384, 0.12131398916244507, 0.05448112636804581]]
Accuracy for iterations:  [[0.9871526956558228, 0.9554057717323303, 0.9804420471191406], [0.9890955090522766, 0.9430901408195496, 0.9842643737792969], [0.9900376796722412, 0.9471408724784851, 0.9879401922225952]]
F1 for iterations:  [[0.9871565102028913, 0.955312338388428, 0.9804419468335733], [0.9890998387839542, 0.9428067404328064, 0.9842643431643833], [0.990042161736056, 0.9468745067838977, 0.9879397635678289]]
Precision for iterations:  [[0.9872243578805475, 0.9556615380304949, 0.9804547372047389], [0.9892309150472579, 0.944483742605825, 0.984269025597288], [0.990217253956467, 0.9486164158111176, 0.9880125896412548]]
Recall for iterations:  [[0.9871527067691139, 0.9554057981544721, 0.9804420760415431], [0.9890955038125566, 0.9430901692944852, 0.9842643807238664], [0.9